This notebook managed to see why FSP3 is having different ranking coefficients in ridge regression. Conclusion is the "drop_duplicates()" function during data preprocessing. If apply, FSP3 will have low rank, (~1st), among other features. If don't apply, FSP3 will rank ~40th among all the features. 

Also, we're using coefficients from unbalanced ridge regression model.

In [2]:
import pandas as pd
import numpy as np
import os
import glob
import sklearn
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import f1_score


In [41]:
df = pd.read_csv('Unbalanced weight feature coefficients.csv')
df.head()

,Unnamed: 0,tox21-ahr-p1,tox21-ap1-agonist-p1,tox21-ar-bla-antagonist-p1,tox21-ar-mda-kb2-luc-agonist-p3,tox21-ar-mda-kb2-luc-antagonist-p1,tox21-ar-mda-kb2-luc-antagonist-p2,tox21-are-bla-p1,tox21-aromatase-p1,tox21-car-agonist-p1,...,tox21-sbe-bla-antagonist-p1,tox21-shh-3t3-gli3-agonist-p1,tox21-shh-3t3-gli3-antagonist-p1,tox21-vdr-bla-agonist-p1,tox21-vdr-bla-antagonist-p1,tox21-dt40-p1_653,tox21-dt40-p1_100,tox21-dt40-p1_657,tox21-rt-viability-hepg2-p1,tox21-rt-viability-hek293-p1
0,MaxEStateIndex,9.489873,0.010173,0.391269,-1.066738,-7.733873,-4.421490,-0.889548,2.615670,-6.402276,...,-0.194472,0.616474,0.251191,-8.263610,-0.101010,-0.041469,-2.802834,-0.013729,-13.427824,-13.511251
1,MinEStateIndex,0.264417,0.010045,-0.212452,0.213937,0.331497,0.101010,-0.333417,-0.151118,0.043820,...,-0.254946,0.135158,0.028295,-0.170409,-0.002924,-0.157037,-0.173553,-0.161387,-0.121827,0.068325
2,MaxAbsEStateIndex,-9.622147,0.010241,-0.456606,0.770614,7.639520,4.013011,0.749963,-2.894077,6.039042,...,-0.172308,-0.789419,-0.493467,8.115939,-0.077401,-0.092006,2.789866,-0.070898,13.002946,13.152236
3,MinAbsEStateIndex,0.263481,0.031418,0.416493,0.273043,0.116204,0.173443,0.510159,0.483448,0.164647,...,0.165480,0.405128,0.211125,0.372329,0.231508,0.209413,0.144352,0.195083,0.074438,0.262263
4,qed,0.353164,0.005052,0.053316,0.107828,0.130901,0.173092,0.223397,0.157245,0.257599,...,0.028008,0.152850,0.056472,0.085101,0.120652,-0.209152,-0.227427,-0.133645,-0.111606,-0.224428


In [42]:
df_copy = df.copy()
df_copy = df_copy.set_index('Unnamed: 0')
df_copy.rank(ascending=False).iloc[101,:]

tox21-ahr-p1                           29.0
tox21-ap1-agonist-p1                   16.0
tox21-ar-bla-antagonist-p1            112.0
tox21-ar-mda-kb2-luc-agonist-p3        22.0
tox21-ar-mda-kb2-luc-antagonist-p1     30.0
tox21-ar-mda-kb2-luc-antagonist-p2     30.0
tox21-are-bla-p1                       50.0
tox21-aromatase-p1                     49.0
tox21-car-agonist-p1                   32.0
tox21-car-antagonist-p1                 3.0
tox21-elg1-luc-agonist-p1             191.0
tox21-er-bla-antagonist-p1            198.0
tox21-er-luc-bg1-4e2-agonist-p4        12.0
tox21-er-luc-bg1-4e2-antagonist-p1      3.0
tox21-er-luc-bg1-4e2-antagonist-p2    204.0
tox21-erb-bla-antagonist-p1             7.0
tox21-erb-bla-p1                        6.0
tox21-err-p1                           15.0
tox21-esre-bla-p1                      45.0
tox21-fxr-bla-agonist-p2              186.0
tox21-fxr-bla-antagonist-p1             7.0
tox21-gh3-tre-antagonist-p1            55.0
tox21-gr-hela-bla-antagonist-p1 

### Pick one assay

In [35]:
one_assay = pd.read_csv("one_assay.csv").drop(one_assay.columns[0],axis=1)
one_assay

,tox21-ahr-p1,SMILES,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,1,[Br-].CCCCCCCCCCCCCC[n+]1ccccc1,2.287998,0.000000,2.287998,0.000000,0.377974,356.392,322.120,355.187462,...,0,0,0,0,0,0,0,0,10,0
1,0,C[C@]12CC[C@H](C[C@H]1O)C2(C)C,9.814236,-0.031250,9.814236,0.031250,0.566800,154.253,136.109,154.135765,...,0,0,0,0,0,0,0,0,0,0
2,0,CCC2=CN(c1ccccc1)C(CCC)C(=C2)CC,2.473380,0.534907,2.473380,0.534907,0.689255,255.405,230.205,255.198700,...,0,0,0,0,0,0,0,0,0,0
3,0,OCC(=O)OCCCC,10.214402,-0.538472,10.214402,0.426481,0.442392,132.159,120.063,132.078644,...,0,0,0,0,0,0,0,0,1,0
4,0,Nc1ccc(cc1)C(=O)OCC,11.092937,-0.307870,11.092937,0.307870,0.532618,165.192,154.104,165.078979,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9679,0,O=S(=O)(O)O.C[C@@H](CC)CCCCC(=O)N[C@@H](CCN)C(...,14.424310,-4.666667,14.424310,0.013058,0.024478,1267.561,1164.745,1266.732957,...,0,0,0,0,0,0,0,0,1,0
9680,0,Cl.O=S(N)(=O)c1ccc(CN)cc1,10.779027,-3.568620,10.779027,0.000000,0.750448,222.697,211.609,222.022976,...,0,1,0,0,0,0,0,0,0,0
9681,0,Oc1cc(cc(O)c1)C(O)CNC(C)C.O=S(=O)(O)O.CC(C)NCC...,9.733565,-4.666667,9.733565,0.047595,0.224312,520.601,484.313,520.209066,...,0,0,0,0,0,0,0,0,0,0
9682,0,[Na+].[O-]C(=O)CCC(=O)OCC(=O)[C@@]4(O)CC[C@@H]...,13.079510,-1.724403,13.079510,0.000000,0.321723,484.521,451.257,484.207312,...,0,0,0,0,0,0,0,0,0,0


### duplicates in first 1000 rows

In [51]:
one_assay.iloc[:1000,:][one_assay.duplicated(keep=False)]

/var/folders/mw/vsqjplld57v73jz2c0hw5x00llg6y6/T/ipykernel_11942/612395730.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  one_assay.iloc[:1000,:][one_assay.duplicated(keep=False)]


,tox21-ahr-p1,SMILES,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
4,0,Nc1ccc(cc1)C(=O)OCC,11.092937,-0.307870,11.092937,0.307870,0.532618,165.192,154.104,165.078979,...,0,0,0,0,0,0,0,0,0,0
7,0,C1N3CN2CN(CN1C2)C3,2.468750,1.187500,2.468750,1.187500,0.426761,140.190,128.094,140.106196,...,0,0,0,0,0,0,0,0,0,0
9,0,CCSCCSP(=S)(OCC)OCC,5.507495,-2.022856,5.507495,0.644046,0.469026,274.413,255.261,274.028479,...,1,0,0,0,0,0,0,0,2,0
11,1,CC(C)[C@@]5(NC(=O)[C@@H]1C[C@H]2[C@@H](CC3=CNC...,14.629783,-2.044999,14.629783,0.164013,0.408639,611.743,570.415,611.310769,...,0,0,0,0,0,0,0,0,0,0
12,0,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,6.112705,-1.456687,6.112705,0.382377,0.540334,354.491,345.419,351.914689,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,0,CCC1(C(=O)NCNC1=O)c2ccccc2,11.986207,-1.069931,11.986207,0.214534,0.711645,218.256,204.144,218.105528,...,0,0,0,0,0,0,0,0,0,0
988,0,O=C(O)C(=O)O,9.099537,-1.824074,9.099537,1.824074,0.379949,90.034,88.018,89.995309,...,0,0,0,0,0,0,0,0,0,0
989,0,Nc1cc(Cl)ccc1,5.564969,0.685185,5.564969,0.685185,0.529786,127.574,121.526,127.018877,...,0,0,0,0,0,0,0,0,0,0
990,0,Clc1cc(C)c(OCC(=O)O)cc1,10.199797,-0.991343,10.199797,0.330881,0.813116,200.621,191.549,200.024022,...,0,0,0,0,0,0,0,0,0,0


In [38]:
one_assay.drop_duplicates()

,tox21-ahr-p1,SMILES,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,1,[Br-].CCCCCCCCCCCCCC[n+]1ccccc1,2.287998,0.000000,2.287998,0.000000,0.377974,356.392,322.120,355.187462,...,0,0,0,0,0,0,0,0,10,0
1,0,C[C@]12CC[C@H](C[C@H]1O)C2(C)C,9.814236,-0.031250,9.814236,0.031250,0.566800,154.253,136.109,154.135765,...,0,0,0,0,0,0,0,0,0,0
2,0,CCC2=CN(c1ccccc1)C(CCC)C(=C2)CC,2.473380,0.534907,2.473380,0.534907,0.689255,255.405,230.205,255.198700,...,0,0,0,0,0,0,0,0,0,0
3,0,OCC(=O)OCCCC,10.214402,-0.538472,10.214402,0.426481,0.442392,132.159,120.063,132.078644,...,0,0,0,0,0,0,0,0,1,0
4,0,Nc1ccc(cc1)C(=O)OCC,11.092937,-0.307870,11.092937,0.307870,0.532618,165.192,154.104,165.078979,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9677,0,Clc1ccc(OCC(O)COC(N)=O)cc1,10.248885,-0.925423,10.248885,0.008842,0.816060,245.662,233.566,245.045486,...,0,0,0,0,0,0,0,0,0,0
9679,0,O=S(=O)(O)O.C[C@@H](CC)CCCCC(=O)N[C@@H](CCN)C(...,14.424310,-4.666667,14.424310,0.013058,0.024478,1267.561,1164.745,1266.732957,...,0,0,0,0,0,0,0,0,1,0
9680,0,Cl.O=S(N)(=O)c1ccc(CN)cc1,10.779027,-3.568620,10.779027,0.000000,0.750448,222.697,211.609,222.022976,...,0,1,0,0,0,0,0,0,0,0
9681,0,Oc1cc(cc(O)c1)C(O)CNC(C)C.O=S(=O)(O)O.CC(C)NCC...,9.733565,-4.666667,9.733565,0.047595,0.224312,520.601,484.313,520.209066,...,0,0,0,0,0,0,0,0,0,0


### FSP3 coefficient rank after dropping duplicates

In [43]:
RANDOM_SEED = 101
np.random.seed(RANDOM_SEED)

In [46]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,LabelBinarizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split

one_assay_drop = one_assay.drop_duplicates()
X = np.array(one_assay_drop.iloc[:,2:])
y = one_assay_drop.iloc[:,0]
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
    
    #Use scaler fit to training data to transform testing data
X_test = scaler.transform(X_test)
    
    # -- Training
logistic_cv_unbalanced = LogisticRegressionCV(penalty='l2',Cs=10,max_iter=10000,class_weight=None).fit(X_train, y_train)
logistic_cv_balanced = LogisticRegressionCV(penalty='l2',Cs=10,max_iter=10000,class_weight='balanced').fit(X_train, y_train)
    

In [48]:
coef_df = pd.DataFrame({'PROTOCOL_NAME':one_assay.columns[0],
                        'Feature':one_assay.columns[2:], 
                        'Unbalanced_coef':logistic_cv_unbalanced.coef_[0],
                        'Balanced_coef':logistic_cv_balanced.coef_[0]}).sort_values(by="Unbalanced_coef")

coef_df['ranks'] = coef_df['Unbalanced_coef'].abs().groupby(coef_df['PROTOCOL_NAME']).rank(ascending=False)
coef_df[coef_df['Feature']=="FractionCSP3"]

,PROTOCOL_NAME,Feature,Unbalanced_coef,Balanced_coef,ranks
101,tox21-ahr-p1,FractionCSP3,-0.545812,-1.247681,1.0


### FSP3 coefficient ranking without dropping duplicates

In [49]:
X = np.array(one_assay.iloc[:,2:])
y = one_assay.iloc[:,0]
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
    
    #Use scaler fit to training data to transform testing data
X_test = scaler.transform(X_test)
    
    # -- Training
logistic_cv_unbalanced = LogisticRegressionCV(penalty='l2',Cs=10,max_iter=10000,class_weight=None).fit(X_train, y_train)
logistic_cv_balanced = LogisticRegressionCV(penalty='l2',Cs=10,max_iter=10000,class_weight='balanced').fit(X_train, y_train)
    
    
coef_df = pd.DataFrame({'PROTOCOL_NAME':one_assay.columns[0],
                        'Feature':one_assay.columns[2:], 
                        'Unbalanced_coef':logistic_cv_unbalanced.coef_[0],
                        'Balanced_coef':logistic_cv_balanced.coef_[0]}).sort_values(by="Unbalanced_coef")

coef_df['ranks'] = coef_df['Unbalanced_coef'].abs().groupby(coef_df['PROTOCOL_NAME']).rank(ascending=False)
coef_df[coef_df['Feature']=="FractionCSP3"]

,PROTOCOL_NAME,Feature,Unbalanced_coef,Balanced_coef,ranks
101,tox21-ahr-p1,FractionCSP3,-1.191138,-1.183267,40.0


### balanced model performance without dropping duplicates

In [50]:
y_pred = logistic_cv_balanced.predict(X_test)
yproba = logistic_cv_balanced.predict_proba(X_test)
# performance
print(f'Accuracy Score: {accuracy_score(y_test,y_pred)}')
print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred)}')
print(f'R^2 square: {sklearn.metrics.r2_score(y_test,y_pred)}')

Accuracy Score: 0.8492514197212184
Confusion Matrix: 
[[1539  253]
 [  39  106]]
Area Under Curve: 0.7949257235221676
Recall score: 0.7310344827586207
R^2 square: -1.1767395320197043
